1. Could you construct a simple playbook to install Nginx on a server?

- hosts: stagingwebservers
 gather_facts: False
 vars:
  - server_port: 8080
 tasks:
  - name: install nginx
    apt: pkg=nginx state=installed update_cache=true
  - name: serve nginx config
     template: src=../files/flask.conf dest=/etc/nginx/conf.d/
     notify:
     - restart nginx
 handlers:
   - name: restart nginx
     service: name=nginx state=restarted
   - name: restart flask app
     service: name=flask-demo state=restarted
            
            
After executing the above playbook we can verify whether Nginx is installed or not.

ps waux | grep nginx

2. What is the significance of the " notation? And how may variables or dynamic variable names be interpolated?

One basic rule is to ‘always use {{}} except when:’. Conditionals are always run through Jinja2 as to resolve the expression. Therefore, ‘when:failed_when:’ and ‘changed_when:’ are always templated and we should avoid adding {{}}.
In other cases, except when clause, we have to use brackets, otherwise, differentiating between an undefined variable and a string will be difficult to do.

3. What is the difference between an Ansible role and a playbook role?

Role is a set of tasks and additional files to configure host to serve for a certain role. Playbook is a mapping between hosts and roles.

4. How can I write a multi-task Ansible handler in Ansible?

Handlers can “listen” to generic topics, and tasks can notify those topics as shown below. This functionality makes it much easier to trigger multiple handlers. It also decouples handlers from their names, making it easier to share handlers among playbooks and roles:

- name: Check if restarted
shell: check_is_started.sh
register: result
listen: Restart processes
 
- name: Restart conditionally step 2
service: name=service state=restarted
when: result
listen: Restart processes

5. What are Ansible Vaults and how do you use them?

Ansible Vault is a feature that allows you to keep all your secrets safe. It can encrypt entire files, entire YAML playbooks or even a few variables. It provides a facility where you can not only encrypt sensitive data but also integrate them into your playbooks.

Vault is implemented with file-level granularity where the files are either entirely encrypted or entirely unencrypted. It uses the same password for encrypting as well as for decrypting files which makes using Ansible Vault very user-friendly

6. How can I use Ansible to create encrypted files?

To create an encrypted file, use the ‘ansible-vault create’ command and pass the filename.

$ ansible-vault create filename.yaml

You’ll be prompted to create a password and then confirm it by re-typing it.

Once your password is confirmed, a new file will be created and will open an editing window. By default, the editor for Ansible Vault is vi. You can add data, save and exit.



7. What is Ansible Tower, exactly?

Ansible Tower is Ansible at a more enterprise level. It is a web-based solution for managing your organization with a very easy user interface that provides a dashboard with all of the state summaries of all the hosts, allows quick deployments, and monitors all configurations.

8. What are the benefits of the Ansible Tower?

Ansible Tower allows you to control access to who can access what, even allowing sharing of SSH credentials without someone being able to transfer those credentials. Inventory can be graphically managed or synced with a wide variety of cloud sources.

Below are some benefits of Ansible Tower

1. Support of Cluster Environment
2. Credentials Storing & using in a Smart Manner
3. CI/CD Workflow Support
4. Job Scheduling and Notification Support
5. Working on a Team 


9. What is the role of Ansible in the Continuous Delivery pipeline? Explain.

It is well known that in DevOps development and operations work is integrated. This integration is very important for modern test-driven applications. Hence, Ansible integrates this by providing a stable environment to both development and operations resulting in a smooth delivery pipeline. 

When developers begin to think of infrastructure as part of their application i.e as Infrastructure as code (IaC), stability and performance become normative. Infrastructure as Code is the process of managing and provisioning computing infrastructure and their configuration through machine-processable definition files, rather than physical hardware configuration or the use of interactive configuration tools. This is where Ansible automation plays a major role and stands out among its peers.

In a Continuous Delivery pipeline, Sysadmins work tightly with developers, development velocity is improved, and more time is spent doing activities like performance tuning, experimenting, and getting things done, and less time is spent fixing problems.

10. Using Ansible, how do you build a LAMP stack and deploy a webpage?

Suppose you’re trying to deploy a website on multiple systems, every website deployment will require a base OS, web-server, Database, and PHP. We use ansible playbook to install these prerequisites on all the systems at once.

We can use two virtual machines, one as a server where Ansible is installed and the other machine acts as the remote host.

Below Ansible playbook has the code to install Apache, MySql, and PHP:


#### Setup LAMP Stack
-  hosts: host1
   tasks:
 
      -  name: Add ppa repository
         become: yes
         apt_repository: repo=ppa:ondrej/php
 
      -  name: Install lamp stack
         become: yes
         apt:
            pkg:
              - apache2   
              - mysql-server
              - php7.0
              - php7.0-mysql   
            state: present
            update cache: yes
 
      -  name: start apache server
          become: yes
          service: 
              name: apache2
               state: started
                enabled: yes
 
      -  name: start mysql service
          become: yes
          services:
             name: mysql
             state: started
             enabled: yes
 
      -  name:  create target directory
          file: path=/var/www/html state=directory mode=0755
 
      - name:  deploy index.html
         became: yes
          copy: 
              src: /etc/ansible/index/index.html
              dest: var/www/html/index/index.html
                
Now, there are 6 main tasks, each task performs a specific function:
    
The first task adds the repository required to install MySQL and PHP.
The second task installs apache2, MySQL-server, PHP, and PHP-MySQL.
The third and fourth task starts the Apache and MySQL service.
The fifth task creates a target directory in the host machine and
Finally, the sixth task executes the index.html file, it picks up the file from the server machine and copies it into the host machine.
To finally run this playbook the following command can be used:

$ ansible-playbook lamp.yml -K